In [1]:
import numpy as np
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
import datetime
from scipy.interpolate import griddata
import tqdm
from regrid import regrid
import matplotlib.pyplot as plt
import xarray as xr
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [2]:
datapath = '/Users/carmennab/Dropbox/OI_PolarSnow/data/CDS_icetype'

days = np.arange(0,4018)
dates = [datetime.date(2010,1,1)+datetime.timedelta(days=int(days)) for days in days]
ease_lats = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lat_25km.npy')
ease_lons = np.load('/Users/carmennab/Dropbox/OI_PolarSnow/EASE/auxiliary/new_lon_25km.npy')

icetype_grid_25 = np.full((len(days),360,360),np.nan)

for day in tqdm.tqdm(days):
    date = dates[day].strftime('%Y%m%d')
    try:
        icetype = Dataset('/Users/carmennab/Dropbox/OI_PolarSnow/data/CDS_icetype/ice_type_nh_ease2-250_cdr-v2p0_' + date + '1200.nc')
        icetype_lon = np.array(icetype['lon'])
        icetype_lat = np.array(icetype['lat'])
        itype = np.array(icetype['ice_type']).astype('float32')[0,:,:]
        itype[itype>3] = np.nan
        itype[itype<2] = np.nan
        icetype_grid_25[day,:,:] = regrid(itype,icetype_lon,icetype_lat,ease_lons,ease_lats)
    except:
        pass

100%|███████████████████████████████████████| 4018/4018 [10:35<00:00,  6.32it/s]


In [3]:
# create icetype NETCDF
data_vars = {'Ice Type':(['t','x','y'],icetype_grid_25)}
ds = xr.Dataset(data_vars = data_vars,
                coords={'Longitude':(['x','y'],ease_lons),
                        'Latitude':(['x','y'],ease_lats),
                         'Day':(['t'],days)}     )
ds.to_netcdf(f'/Users/carmennab/Dropbox/OI_PolarSnow/EASE/data/icetype.nc','w')